# Set up

In [2]:
import sys

import numpy as np
from dotenv import load_dotenv

load_dotenv()

sys.path.insert(0, "..")

from src.math_utils import sigmoid
from src.model import Item2ItemCollaborativeFiltering, User2UserCollaborativeFiltering

# U2U

In [3]:
# Mock data
user_indices = [0, 0, 0, 1, 1, 2, 2, 2, 3, 3]
item_indices = [0, 2, 3, 1, 4, 0, 1, 2, 3, 4]
ratings = [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
n_users = len(set(user_indices))
n_items = len(set(item_indices))

val_user_indices = [0, 2, 1]
val_item_indices = [1, 3, 2]
val_ratings = [5, 5, 5]

print("Mock User IDs:", user_indices)
print("Mock Item IDs:", item_indices)
print("Ratings:", ratings)


Mock User IDs: [0, 0, 0, 1, 1, 2, 2, 2, 3, 3]
Mock Item IDs: [0, 2, 3, 1, 4, 0, 1, 2, 3, 4]
Ratings: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
[0.5 0.5 0.5]


In [ ]:
model = User2UserCollaborativeFiltering(n_users, n_items)

users = val_user_indices
items = val_item_indices
predictions = model.predict(users, items)
print(predictions)

In [4]:
model.fit(user_indices, item_indices, ratings)
predictions = model.predict(users, items)
print(predictions)

[0.99330715 0.99330715 0.99330715]


#### 🧐 Go into details

In [5]:
model.user_item_matrix

array([[5., 0., 5., 5., 0.],
       [0., 5., 0., 0., 5.],
       [5., 5., 5., 0., 0.],
       [0., 0., 0., 5., 5.]])

In [6]:
model.user_similarity

array([[0.        , 0.        , 0.66666667, 0.40824829],
       [0.        , 0.        , 0.40824829, 0.5       ],
       [0.66666667, 0.40824829, 0.        , 0.        ],
       [0.40824829, 0.5       , 0.        , 0.        ]])

In [7]:
# Inspect one example
user = 0
item = 1

sim_scores = model.user_similarity[user]
print(f"{sim_scores=}")

sim_scores=array([0.        , 0.        , 0.66666667, 0.40824829])


In [8]:
# Only consider users who have rated the item
user_ratings = model.user_item_matrix[:, item]
print(f"{user_ratings=}")
sim_scores = sim_scores[user_ratings != 0]
print(f"{sim_scores=}")
user_ratings = user_ratings[user_ratings != 0]
print(f"{user_ratings=}")

user_ratings=array([0., 5., 5., 0.])
sim_scores=array([0.        , 0.66666667])
user_ratings=array([5., 5.])


In [9]:
# Weighted average of ratings
print(f"Weighted average: {np.dot(sim_scores, user_ratings)}")
print(f"Normalization factor: {np.sum(sim_scores)}")
print(f"Predicted rating: {np.dot(sim_scores, user_ratings) / np.sum(sim_scores)}")
print(
    f"Predicted rating - sigmoid: {sigmoid(np.dot(sim_scores, user_ratings) / np.sum(sim_scores))}"
)

Weighted average: 3.333333333333333
Normalization factor: 0.6666666666666666
Predicted rating: 5.0
Predicted rating - sigmoid: 0.9933071490757153


In [11]:
recommendations = model.recommend(val_user_indices, k=2)

Generating recommendations:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
recommendations

{'user_indice': [np.int64(0),
  np.int64(0),
  np.int64(2),
  np.int64(2),
  np.int64(1),
  np.int64(1)],
 'recommendation': [np.int64(4),
  np.int64(3),
  np.int64(4),
  np.int64(3),
  np.int64(4),
  np.int64(3)],
 'score': [np.float64(0.9933071490757153),
  np.float64(0.9933071490757153),
  np.float64(0.9933071490757153),
  np.float64(0.9933071490757153),
  np.float64(0.9933071490757153),
  np.float64(0.9933071490757153)]}

# I2I

In [13]:
print("Mock User IDs:", user_indices)
print("Mock Item IDs:", item_indices)
print("Ratings:", ratings)

model = Item2ItemCollaborativeFiltering(n_users, n_items)

predictions = model.predict(users, items)
print(predictions)

Mock User IDs: [0, 0, 0, 1, 1, 2, 2, 2, 3, 3]
Mock Item IDs: [0, 2, 3, 1, 4, 0, 1, 2, 3, 4]
Ratings: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
[0.5 0.5 0.5]


In [14]:
model.fit(user_indices, item_indices, ratings)
predictions = model.predict(users, items)
print(predictions)

[0.99330715 0.99330715 0.99330715]


#### 🧐 Go into details

In [15]:
model.user_item_matrix

array([[5., 0., 5., 5., 0.],
       [0., 5., 0., 0., 5.],
       [5., 5., 5., 0., 0.],
       [0., 0., 0., 5., 5.]])

In [16]:
model.item_similarity

array([[0. , 0.5, 1. , 0.5, 0. ],
       [0.5, 0. , 0.5, 0. , 0.5],
       [1. , 0.5, 0. , 0.5, 0. ],
       [0.5, 0. , 0.5, 0. , 0.5],
       [0. , 0.5, 0. , 0.5, 0. ]])